# 1. 概述

条件随机场（Conditional Random Field, CRF）本质上是概率图模型（Probabilistic Graphical Model）与区分性分类( Discriminative Classification）的一种接合，能够用来对序列分类（标注）问题进行建模。

论文 [1] 中经典的图（图1），阐释了 CRF 与其他模型之间的关系。

![](http://ww3.sinaimg.cn/large/6cbb8645jw1en4lqd5qq3j21510p1jvn.jpg)
**图1.【[src](http://www.hankcs.com/nlp/segment/crf-segmentation-of-the-pure-java-implementation.html)】**


The advantage to a conditional model is that dependencies that involve only variables in x play no role in the conditional model, so that an accurate conditional model can have much simpler structure than a joint model.

![]()
**图2. 线性链 CRF**

下面我们关注线性链条件随机场（Linear-Chain CRF）（如图2）的原理与实现。线性链 CRF 通过与双向 LSTM（Bi-LSTM）的接合，我们图3关系的更一般 CRF。 

![]()
**图3. 线性链 CRF**

# 2. 模型形式化

http://www.cs.columbia.edu/~mcollins/fb.pdf

![](https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=1645229592,3117226322&fm=27&gp=0.jpg)
**线性链条件随机场**

给定长度为 $m$ 的序列， 以及状态集 $S$。 对于任意状态序列 $(s_1,\cdots,s_m), s_i \in S$, 定义其“势”（potential）如下：
$$
\psi(s_1,\dots, s_m) = \prod_{i=1}^m\psi (s_{i−1}, s_i , i)
$$
我们定义 $s_0$ 为特殊的开始符号 $*$。这里对 $s, s^\prime ∈ S, i \in {1 , \dots, m}$，势函数 $\psi(s, s^\prime, i) \ge 0$。也即，势函数是非负的，它对序列第 $i$ 位置发生的 $s$ 到 $s^\prime$ 的状态转移都给出一个非负值。



根据概率图模型的因子分解理论[?]，我们有：
$$
p(s_1,\dots,s_m|x_1,\dots, x_m) = \frac{\psi(s_1,\dots, s_m) }{\sum_{s^\prime_1,\dots,s^\prime_m} \psi(s^\prime_1,\dots, s^\prime_m)}
$$

$Z = \sum_{s^\prime_1,\dots,s^\prime_m} \psi(s^\prime_1,\dots, s^\prime_m) $ 为归一化因子。



# 算法

同 HMM 类似，CRF 也涉及三类基本问题：评估（计算某一序列的似然值）、解码（给定输入，寻找似然最大的序列）及训练（根据数据估计 CRF 的参数），解决这三个问题也都涉及前向算法、后向算法及 Viterbi 算法。

CRF 的势函数类似于概率，只不过没有归一化，因此这里介绍的 CRF 前向算法、Viterbi 算法、后向算法，同 HMM 基本一致。

## 2.1 前向算法

定义：
$$
\alpha(i, s) = \sum_{s_1,\dots,s_{i-1}} \psi(s_1,\dots,s_{i-1}, s)
$$

表示，以 $s$ 结尾的长度为 $i$ 的子序列的势。

显然，$\alpha(1, s) = \psi(*, s_1, 1)$

根据定义，我们有如下递归关系：
$$
\alpha(i, s) = \sum_{s^\prime \in S} \alpha(i-1, s^\prime) \times \psi(s^\prime, s, i)
$$

归一化因子可以计算如下：
$$Z = \sum_{s_1,\dots,s_m} \psi(s_1,\dots s_m) = \sum_{s\in S}\sum_{s_1,\dots,s_{m-1}} \psi(s_1,\dots s_{m-1}, s)= \sum_{s\in S} \alpha(m, s)
$$

对于给定的序列 $(s_1,\cdots,s_m)$，其中条件概率（似然）可以计算：
$$
p(s_1,\dots,s_m|x_1,\dots, x_m) = \frac{\prod_{i=1}^m\psi (s_{i−1}, s_i , i)}{\sum_{s\in S} \alpha(m, s)}
$$

** 通过前向算法，我们解决了评估问题，计算和空间复杂度为 $O(m\cdot|S|^2)$。**

> 似然的计算过程中，只涉及乘法和加法，都是可导操作。因此，只需要实现前向操作，我们就可以借具有自动梯度功能的学习库（e.g. pytorch、tensorflow）实现基于最大似然准则的训练。

In [1]:
import numpy as np

def forward(psi):
    m, V, _ = psi.shape
    
    alpha = np.zeros([m, V])
    alpha[0] = psi[0, 0, :]  # assume psi[0, 0, :] := psi(*,s,1)
    
    for i in range(1, m):
        for j in range(V):
            '''
            for k in range(V):
                alpha[i, j] += alpha[i - 1, k] * psi[i, k, j]
            '''
            alpha[i, j] = np.sum(alpha[i - 1, :] * psi[i, :, j])
                
    return alpha

def pro(seq, psi):
    m, V, _ = psi.shape
    alpha = forward(psi)
    
    Z = np.sum(alpha[-1])
    M = psi[0, 0, seq[0]]
    for i in range(1, m):
        M *= psi[i, seq[i-1], seq[i]]
        
    p = M / Z
    return p

np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

alpha = forward(psi)
p = pro(seq, psi)
print(p)
print(alpha)

2.69869828108e-08
[[  1.10026295e+00   2.52187760e+00   1.40997704e+00   1.36407554e+00
    1.00201186e+00]
 [  1.27679086e+01   1.03890052e+01   1.44699134e+01   1.15244329e+01
    1.52767179e+01]
 [  9.30306192e+01   1.09450375e+02   1.26777728e+02   1.28529576e+02
    1.16835669e+02]
 [  9.81861108e+02   8.70384204e+02   9.35531558e+02   7.98228277e+02
    9.89225754e+02]
 [  6.89790063e+03   8.71016058e+03   8.84778486e+03   9.21051594e+03
    6.56093883e+03]
 [  7.56109978e+04   7.00773298e+04   8.60611103e+04   5.63567069e+04
    5.99238226e+04]
 [  6.69236243e+05   6.42107210e+05   7.81638452e+05   6.32533145e+05
    5.71122492e+05]
 [  6.62242340e+06   5.24446290e+06   5.54750409e+06   4.68782248e+06
    4.49353155e+06]
 [  4.31080734e+07   4.09579660e+07   4.62891972e+07   4.60100937e+07
    4.63083098e+07]
 [  2.66620185e+08   4.91942550e+08   4.48597546e+08   3.42214705e+08
    4.10510463e+08]]


## 2.2 Viterbi 解码

Viterbi 利用动态规划，寻找似然最大的序列。Viterbi 与前向算法非常相似，只是将求和操作替换为最大值操作。

$$
\alpha(j, s) = \underset{s_1,\dots,s_{j-1}}{\mathrm{max}}\psi(s_1,\dots,s_{j-1}, s)
$$
显然，$\alpha(1, s) = \psi(*, s_1, 1)$

根据定义，我们有如下递归关系：
$$
\alpha(j, s) = \underset{s^\prime \in S}{\mathrm{max}}\  \alpha(j-1, s^\prime) \cdot \psi(s^\prime, s, j)
$$

在所有 $|s|^m$ 条可能的序列中，概率最大的路径的未归一化的值为：
$$
\max \alpha(m, s)
$$
沿着前向推导的反方向，可以得到最优的路径，算法复杂度是 $O(m*|S|^2)$。

In [2]:
def viterbi(psi):
    m, V, _ = psi.shape
    
    alpha = np.zeros([m, V])
    trans = np.ones([m, V]).astype('int') * -1

    alpha[0] = psi[0, 0, :]  # assume psi[0, 0, :] := psi(*,s,1)
    
    for i in range(1, m):
        for j in range(V):
            tmp = alpha[i - 1, :] * psi[i, :, j]
            alpha[i, j] = np.max(tmp)
            trans[i, j] = np.argmax(tmp)
            
    end = np.argmax(alpha[-1])
    path = [end]
    for t in range(m - 1, 0, -1):
        cur = path[-1]
        pre = trans[t, cur]
        path.append(pre)

    return path[::-1]

np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative

path = viterbi(psi)
print(path)

[1, 4, 2, 4, 3, 0, 3, 0, 3, 1]


## 2.3 后向算法

为了训练 CRF, 我们需要计算相应的梯度。有了高效的手动计算梯度，需要用到后向算法。

定义：

$$
\beta(j, s) = \sum_{s_{j+1},\dots, s_m} \psi(s_{j+1},\dots, s_m|s_j=s)
$$

其中，令 $\beta(m, s) = 1$

根据定义，我们有如下递归关系：

$$
\beta(j, s) = \sum_{s^\prime \in S} \beta(j+1, s^\prime) \cdot \psi(s, s^\prime, j+1)
$$

In [3]:
def backward(psi):
    m, V, _ = psi.shape
    
    beta = np.zeros([m, V])
    beta[-1] = 1
    
    for i in range(m - 2, -1, -1):
        for j in range(V):
            '''
            for k in range(V):
                beta[i, j] += beta[i + 1, k] * psi[i, j, k]
            '''
            beta[i, j] = np.sum(beta[i + 1, :] * psi[i, j, :])
                
    return beta

np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

beta = backward(psi)
print(beta)

[[  2.22673640e+08   2.55691709e+08   2.23296038e+08   2.47196311e+08
    3.09104234e+08]
 [  3.35193570e+07   2.97241970e+07   3.60109028e+07   2.30593845e+07
    2.85460276e+07]
 [  3.10732364e+06   2.63044619e+06   4.11747686e+06   3.23523521e+06
    3.84952526e+06]
 [  3.32665741e+05   3.41459546e+05   4.75111304e+05   3.75856583e+05
    3.92092981e+05]
 [  5.00577045e+04   4.74982617e+04   4.49600924e+04   5.11404121e+04
    4.98358252e+04]
 [  5.12607218e+03   6.14087407e+03   4.97562865e+03   5.66824187e+03
    5.73024401e+03]
 [  7.39391942e+02   5.92830408e+02   6.43990264e+02   5.38430038e+02
    6.58450058e+02]
 [  5.49771223e+01   6.09662863e+01   6.83282385e+01   8.03727866e+01
    7.43932095e+01]
 [  8.59855198e+00   9.03797021e+00   1.02394530e+01   6.64334111e+00
    6.76191475e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    1.00000000e+00]]


## 2.4 梯度计算
$$
Z = \sum_{s_1,\dots,s_m} \psi(s_1,\dots, s_m) = \sum_{s^\prime_{i-1} \in S, s^\prime_i \in S} \sum_{s_{i-1}=s^\prime_{i-1}, s_i=s^\prime_i} \psi(s_1,\dots, s_m) = \sum_{s^\prime_{i-1} \in S, s^\prime_i \in S} \alpha(i-1, s^\prime_{i-1}) \cdot \beta(i, s^\prime_i) \cdot \psi (s^\prime_{i−1}, s^\prime_i , i) \ \ \ 1 < i \le m
$$

对于 $i = 1$ 的边界情况：
$$
Z = \sum_{s^\prime_1 \in S} \beta(1, s^\prime_i) \cdot \psi (*, s^\prime_1 , 1)
$$

对于路径 $(s_1, \cdots, s_m)$，
$$
p(s_1,\dots,s_m|x_1,\dots, x_m) = \frac{\psi(s_1,\dots, s_m)}{Z} = \frac{\prod_{i=1}^m\psi (s_{i−1}, s_i , i)}{Z} = \frac{\prod_{i=1}^m\psi^i_{s_{i−1}, s_i}}{Z}
$$
其中，$\psi^i_{s^\prime, s} = \psi(s^\prime, s, i),\ s^\prime, s \in S$。

记分子 $\prod_{i=1}^m\psi (s_{i−1}, s_i , i) = M$ 则：
$$
\frac{\partial p(s_1,\dots,s_m|x_1,\dots, x_m)}{\partial \psi^k_{s^\prime, s}} = \frac{1}{Z} [ \frac{M}{\psi^k_{s^\prime, s}} \cdot \delta_{s^\prime = s_{k-1} \& s = s_k} - p \cdot \alpha(k-1, s^\prime) \cdot \beta(k, s)]
$$

其中，$\delta_{true} = 1, \delta_{false} = 0$。

$$
\frac{\partial \ln p(s_1,\dots,s_m|x_1,\dots, x_m)}{\partial \psi^k_{s^\prime, s}}  = \frac{1}{p} \cdot \frac{\partial p(s_1,\dots,s_m|x_1,\dots, x_m)}{\partial \psi^k_{s^\prime, s}} = \frac{\delta_{s^\prime = s_{k-1} \& s = s_k}}{\psi^k_{s^\prime, s}} - \frac{1}{Z} \alpha(k-1, s^\prime) \cdot \beta(k, s)
$$


In [4]:
def gradient(seq, psi):
    m, V, _ = psi.shape
    
    grad = np.zeros_like(psi)
    alpha = forward(psi)
    beta = backward(psi)
    
    Z = np.sum(alpha[-1])
    
    for k in range(1, m):
        for i in range(V):
            for j in range(V):
                grad[k, i, j] = -alpha[k - 1, i] * beta[k, j] / Z
                
                if i == seq[k-1] and j == seq[k]:
                    grad[k, i, j] += 1. / psi[k, i, j]

    # corner cases
    grad[0, 0, :] = -beta[0, :] / Z
    grad[0, 0, seq[0]] += 1. / psi[0, 0, seq[0]]
    
    return grad
    
np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

grad = gradient(seq, psi)
print(grad[0, :, :])

[[ 0.795258   -0.13046258 -0.11393321 -0.12612794 -0.15771546]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [5]:
def check_grad(seq, psi, i, j, k, toleration=1e-5, delta=1e-10):
    m, V, _ = psi.shape
    
    grad_1 = gradient(seq, psi)[i, j, k]
    
    original = psi[i, j, k]
    
    # p1
    psi[i, j, k] = original - delta
    p1 = np.log(pro(seq, psi))
    
    # p2
    psi[i, j, k] = original + delta
    p2 = np.log(pro(seq, psi))
    
    psi[i, j, k] = original
    grad_2 = (p2 - p1) / (2 * delta)
    
    diff = np.abs(grad_1 - grad_2)
    if diff > toleration:
        print("%d, %d, %d, %.2e, %.2e, %.2e" % (i, j, k, grad_1, grad_2, diff))
    
np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi) # nonnegative
seq = np.random.choice(V, m)
print(seq)

for toleration in [1e-2]:
    print(toleration)
    for i in range(m):
        for j in range(V):
            for k in range(V):
                check_grad(seq, psi, i, j, k, toleration)

[0 1 4 1 3 0 0 3 3 1]
0.01
0, 0, 0, 7.95e-01, 7.58e-01, 3.69e-02
0, 0, 2, -1.14e-01, -1.62e-01, 4.78e-02
0, 0, 3, -1.26e-01, -1.04e-01, 2.26e-02
0, 0, 4, -1.58e-01, -1.28e-01, 2.95e-02
6, 2, 0, -3.25e-02, -2.12e-02, 1.12e-02


首先定义基本的 log 域加法操作（[参见](https://github.com/DingKe/ml-tutorial/blob/master/ctc/CTC.ipynb)）。

In [6]:
ninf = -np.float('inf')

def _logsumexp(a, b):
    '''
    np.log(np.exp(a) + np.exp(b))

    '''

    if a < b:
        a, b = b, a

    if b == ninf:
        return a
    else:
        return a + np.log(1 + np.exp(b - a)) 
    
def logsumexp(*args):
    '''
    from scipy.special import logsumexp
    logsumexp(args)
    '''
    res = args[0]
    for e in args[1:]:
        res = _logsumexp(res, e)
    return res

In [7]:
def forward_log(log_psi):
    m, V, _ = log_psi.shape
    
    log_alpha = np.ones([m, V]) * ninf
    log_alpha[0] = log_psi[0, 0, :]  # assume psi[0, 0, :] := psi(*,s,1)
    
    for i in range(1, m):
        for j in range(V):
            for k in range(V):
                log_alpha[i, k] = logsumexp(log_alpha[i, k], log_alpha[i - 1, j] + log_psi[i, j, k])
                
    return log_alpha

def pro_log(seq, log_psi):
    m, V, _ = log_psi.shape
    log_alpha = forward_log(log_psi)
    
    log_Z = logsumexp(*[e for e in log_alpha[-1]])
    log_M = log_psi[0, 0, seq[0]]
    for i in range(1, m):
        log_M = log_M + log_psi[i, seq[i - 1], seq[i]]
        
    log_p = log_M - log_Z
    return log_p

np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

alpha = forward(psi)
log_alpha = forward_log(log_psi)
print(np.sum(np.abs(np.log(alpha) - log_alpha)))

p = pro(seq, psi)
log_p = pro_log(seq, log_psi)
print(np.sum(np.abs(np.log(p) - log_p)))

3.03719722983e-14
0.0


In [8]:
def backward_log(log_psi):
    m, V, _ = log_psi.shape
    
    log_beta = np.ones([m, V]) * ninf
    log_beta[-1] = 0
    
    for i in range(m - 2, -1, -1):
        for j in range(V):
            for k in range(V):
                log_beta[i, j] = logsumexp(log_beta[i, j], log_beta[i + 1, k] + log_psi[i, j, k])
                
    return log_beta

np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

beta = backward(psi)
log_beta = backward_log(log_psi)

print(np.sum(np.abs(beta - np.exp(log_beta))))
print(np.sum(np.abs(log_beta - np.log(beta))))

2.22687807128e-06
1.64313007645e-14


In [9]:
def gradient_log(seq, log_psi):
    m, V, _ = log_psi.shape
    
    grad = np.zeros_like(log_psi)
    log_alpha = forward_log(log_psi)
    log_beta = backward_log(log_psi)
     
    log_Z = logsumexp(*[e for e in log_alpha[-1]])
    for i in range(1, m):
        for j in range(V):
            for k in range(V):
                grad[i, j, k] -= np.exp(log_alpha[i - 1, j] + log_beta[i, k] - log_Z)
                if j == seq[i - 1] and k == seq[i]:
                    grad[i, j, k] += np.exp(-log_psi[i, j, k])

    # corner cases
    grad[0, 0, :] -= np.exp(log_beta[0, :] - log_Z)
    grad[0, 0, seq[0]] += np.exp(-log_psi[0, 0, seq[0]])
    
    return grad
    
np.random.seed(1111)
V, m = 5, 10

log_psi = np.random.random([m, V, V])
psi = np.exp(log_psi)  # nonnegative
seq = np.random.choice(V, m)

grad_1 = gradient(seq, psi)
grad_2 = gradient_log(seq, log_psi)

print(grad_1[0, :, :])
print(grad_2[0, :, :])
print(np.sum(np.abs(grad_1 - grad_2)))

[[ 0.795258   -0.13046258 -0.11393321 -0.12612794 -0.15771546]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.795258   -0.13046258 -0.11393321 -0.12612794 -0.15771546]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
1.12930498286e-14


# 3. CRF + 人工神经网络

## 3.1 势函数选择


目前为止，我们都假设函数已经知道，在此基础上推导 CRF 的相关计算。理论上，除了非负性的要求 ，势函数可以灵活的选择。为也便于计算和训练，CRF 中一般选择指数的形式。假设输入为 $x_1,\dots,x_m$，则势函数定义为：

$$
\psi(s^\prime, s, i) = \exp(w \cdot \phi(x_1,\dots,x_m, s^\prime, s, i))
$$

则
$$
\psi(s_1,\dots, s_m) = \prod_{i=1}^m\psi (s_{i−1}, s_i , i) = \prod_{i=1}^m\exp(w \cdot \phi(x_1,\dots,x_m, s_{i-1}, s_i,i))
$$

其中，$\phi(x_1,\dots,x_m, s^\prime, s, i) \in R^d$ 是特征向量，$w \in R^d$ 是参数向量。 

对于线性链模型，简化势函数为：
$$
\psi(s^\prime, s, i) = t(s|s^\prime)e(s|x_i)
$$

转移势函数定义为：
$$
t(s|s^\prime) = \exp(v \cdot g(s^\prime, s))
$$

发射势函数定义为：
$$
e(s|x_i) = \exp(w \cdot f(s, x_i))
$$

则：
$$
\psi(s_1,\dots, s_m) = \prod_{j=1}^m\psi (s_{j−1}, s_j , j) = \prod_{j=1}^m t(s_j|s_{j-1})e(s | x_j) = \prod_{j=1}^m  \exp(v \cdot g(s_{j-1}, s_j)) \cdot \exp(w \cdot f(s_j, x_j)) 
$$

$$
\psi(s_1,\dots, s_m) = \exp(\sum_{i=1}^m v \cdot g(s_{i-1}, s_i) + \sum_{i=1}^m w \cdot f(s_i, x_i))
$$

如果我们取对数，则我们得到一个线性模型，定义：

$$
score_t(s|s^\prime) = \log t(s|s^\prime) = v \cdot g(s^\prime, s)
$$

$$
score_e(s|x_i) = \log e(s|x_i) = w \cdot f(s, x_i)
$$

则

$$
\log \psi(s_1,\dots, s_m) = \sum_{i=1}^m v \cdot g(s_{i-1}, s_i) + \sum_{i=1}^m w \cdot f(s_i, x_i) = \sum_{i=1}^m  score_t(s_{i-1}|s_i) + \sum_{i=1}^m score_e(s_i|x_i)
$$

具体的，可以定义
$$
score_t(s_j|s_i) = P_{ij}
$$
其中，$P$ 是 $|S| \times |S|$ 的转移矩阵。

如果 $x = (x_1,\cdots,x_m) \in R^m$，则有： 

$$
score_e(s_j|x_i) = W_j \cdot x_i
$$
其中，$W \in R^{|s| \times n}$ 是权重矩阵。

$$
\log \psi(s_1,\dots, s_m) = \sum_{i=1}^m  score_t(s|s^\prime) + \sum_{i=1}^m score_e(s|x_i) = \sum_{i=1}^m P_{s_{i-1} s_{i}} +  \sum_{i=1}^m W_{s_i} \cdot x_i
$$

这里，为简单起见，我们令 $x_i$ 是一个标量，实际中 $x_i$ 往往是向量。
从 $x$ 到 $\log\psi$ 再到 $\psi$ 都是可导的操作（四则运算和指数、对数运算），而 $\psi$ 的梯度我们上面已经推导可以求得。因此，我们可以利用误差反传计算 $W$ 等参数的梯度，从而利用 SGD 等优化方法训练包括 CRF 在内的整个模型的参数。

In [10]:
def score(seq, x, W, P, S):
    m = len(seq)
    V = len(W)
    
    log_psi = np.zeros([m, V, V])
    
    # corner cases
    for i in range(V):
        # emit
        log_psi[0, 0, i] += S[i]
        # transmit
        log_psi[0, 0, i] += x[0] * W[i]
        
    for k in range(1, m):
        for i in range(V):
            for j in range(V):
                # emit
                log_psi[k, i, j] += x[k] * W[j]
                # transmit
                log_psi[k, i, j] += P[i, j]
    
    return log_psi   

def gradient_param(seq, x, W, P, S):
    m = len(seq)
    V = len(W)
    
    log_psi = score(seq, x, W, P, S)
    
    grad_psi = gradient_log(seq, log_psi)
    grad_log_psi = np.exp(log_psi) * grad_psi
    
    grad_x = np.zeros_like(x)
    grad_W = np.zeros_like(W)
    grad_P = np.zeros_like(P)
    grad_S = np.zeros_like(S)
    
    # corner cases
    for i in range(V):
        # emit
        grad_S[i] += grad_log_psi[0, 0, i]
        # transmit
        grad_W[i] += grad_log_psi[0, 0, i] * x[0]
        grad_x[0] += grad_log_psi[0, 0, i] * W[i]
        
    for k in range(1, m):
        for i in range(V):
            for j in range(V):
                # emit
                grad_W[j] += grad_log_psi[k, i, j] * x[k]
                grad_x[k] += grad_log_psi[k, i, j] * W[j]
                # transmit
                grad_P[i, j] += grad_log_psi[k, i, j]
    
    return grad_x, grad_W, grad_P, grad_S
    
np.random.seed(1111)
V, m = 5, 7

seq = np.random.choice(V, m)
x = np.random.random(m)
W = np.random.random(V)
P = np.random.random([V, V])
S = np.random.random(V)

grad_x, grad_W, grad_P, grad_S = gradient_param(seq, x, W, P, S)

print(grad_x)
print(grad_W)
print(grad_P)
print(grad_S)

[ 0.21877927 -0.07238621  0.00982091  0.09403424  0.00268227  0.16970602
  0.02198856]
[-0.60981665 -0.35102819 -0.16646914 -0.62627954  1.86952017]
[[-0.3357322  -0.26338473 -0.16261862 -0.27312153  0.74243095]
 [-0.21980123 -0.28541281 -0.26073234 -0.25597992  0.87803147]
 [-0.26955437 -0.26679122 -0.32508983 -0.18040223  0.83413031]
 [-0.26161018 -0.16332597 -0.25395322 -0.28057706 -0.2470377 ]
 [ 0.7318243   0.86677824  0.77848947 -0.20103536 -0.10895564]]
[-0.10780192 -0.08705594 -0.09323935 -0.07586051  0.90853183]


梯度正确性检验如下：

In [11]:
def check_grad(seq, x, W, P, S, toleration=1e-5, delta=1e-10):
    m, V, _ = psi.shape
    
    grad_x, grad_W, grad_P, grad_S = gradient_param(seq, x, W, P, S)

    def llk(seq, x, W, P, S):
        log_psi = score(seq, x, W, P, S)
        spi = np.exp(log_psi)
        log_p = np.log(pro(seq, spi))
        return log_p
        
    # grad_x
    print('Check X')
    for i in range(len(x)):
        original = x[i]
        grad_1 = grad_x[i]
        
        # p1
        x[i] = original - delta
        p1 = llk(seq, x, W, P, S)
    
        # p2
        x[i] = original + delta
        p2 = llk(seq, x, W, P, S)
        
        x[i] = original
        grad_2 = (p2 - p1) / (2 * delta)
    
        diff = np.abs(grad_1 - grad_2) / np.abs(grad_2)
        if diff > toleration:
            print("%d, %.2e, %.2e, %.2e" % (i, grad_1, grad_2, diff))
            
    # grad_W
    print('Check W')
    for i in range(len(W)):
        original = W[i]
        grad_1 = grad_W[i]
        
        # p1
        W[i] = original - delta
        p1 = llk(seq, x, W, P, S)
    
        # p2
        W[i] = original + delta
        p2 = llk(seq, x, W, P, S)
        
        W[i] = original
        grad_2 = (p2 - p1) / (2 * delta)
    
        diff = np.abs(grad_1 - grad_2) / np.abs(grad_2)
        if diff > toleration:
            print("%d, %.2e, %.2e, %.2e" % (i, grad_1, grad_2, diff))
            
    # grad_P
    print('Check P')
    for i in range(V):
        for j in range(V):
            original = P[i][j]
            grad_1 = grad_P[i][j]
        
            # p1
            P[i][j] = original - delta
            p1 = llk(seq, x, W, P, S)
    
            # p2
            P[i][j] = original + delta
            p2 = llk(seq, x, W, P, S)
        
            P[i][j] = original
            grad_2 = (p2 - p1) / (2 * delta)
    
            diff = np.abs(grad_1 - grad_2) / np.abs(grad_2)
            if diff > toleration:
                print("%d, %.2e, %.2e, %.2e" % (i, grad_1, grad_2, diff))
                
    # grad_S
    print('Check S')
    for i in range(len(S)):
        original = S[i]
        grad_1 = grad_S[i]
        
        # p1
        S[i] = original - delta
        p1 = llk(seq, x, W, P, S)
    
        # p2
        S[i] = original + delta
        p2 = llk(seq, x, W, P, S)
        
        S[i] = original
        grad_2 = (p2 - p1) / (2 * delta)
    
        diff = np.abs(grad_1 - grad_2) / np.abs(grad_2)
        if diff > toleration:
            print("%d, %.2e, %.2e, %.2e" % (i, grad_1, grad_2, diff))
    
np.random.seed(1111)
V, m = 5, 10

seq = np.random.choice(V, m)
x = np.random.random(m)
W = np.random.random(V)
P = np.random.random([V, V])
S = np.random.random(V)

check_grad(seq, x, W, P, S)


Check X
0, 6.48e-01, 4.61e-01, 4.07e-01
1, 6.50e-02, 6.75e-02, 3.66e-02
2, 5.76e-01, 5.14e-01, 1.19e-01
3, -3.71e-01, -3.17e-01, 1.71e-01
4, 5.06e-01, 4.88e-01, 3.63e-02
5, -2.11e-02, -6.42e-02, 6.72e-01
6, 2.67e-01, 4.19e-01, 3.63e-01
7, -5.65e-01, -4.73e-01, 1.95e-01
8, 2.77e-02, -4.38e-02, 1.63e+00
Check W
0, -7.73e-01, -6.55e-01, 1.81e-01
1, -4.98e-01, -4.52e-01, 1.01e-01
2, -3.16e-02, -1.33e-03, 2.27e+01
3, 6.32e-02, 5.89e-02, 7.44e-02
4, 9.79e-01, 1.05e+00, 6.73e-02
Check P
0, -4.55e-01, -4.51e-01, 9.66e-03
0, -2.76e-01, -2.70e-01, 1.99e-02
0, -3.81e-01, -3.80e-01, 2.87e-03
0, -3.01e-01, -3.02e-01, 2.86e-03
0, 1.59e+00, 1.61e+00, 8.79e-03
1, -2.13e-01, -2.11e-01, 8.42e-03
1, -5.47e-01, -5.37e-01, 1.88e-02
1, -2.67e-01, -2.67e-01, 1.56e-03
1, -2.34e-01, -2.35e-01, 4.32e-03
1, 4.49e-01, 4.68e-01, 4.00e-02
2, -2.96e-01, -2.93e-01, 1.20e-02
2, -1.54e-01, -1.50e-01, 2.13e-02
2, -1.73e-01, -1.72e-01, 4.20e-03
2, -3.48e-01, -3.48e-01, 1.15e-03
2, 5.26e-01, 5.43e-01, 3.23e-02
3, 7.03e-01

## 3.2 Bi-LSTM + CRF
CRF 是强大的序列学习准则。配合双向循环神经网络（e.g. Bi-LSTM）的特征表征和学习能力，在许多序列学习任务上都取得了领先的结果[5~7]。

基本模型如下：
![](https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2428423495,1906169408&fm=27&gp=0.jpg)

Bi-LSTM 对整个输入序列进行特征提取和建模，输出发射。转移势能用另外的 $P$ 表示，作为 CRF 自身的参数。相对于常规的目标函数，CRF 是带参数的训练准则。

对于序列的开头和结尾，可以使用额外的参数，添加。

In [12]:
'''
Pytorch CRF
'''

import torch
from torch.autograd import Function

class CRFF(Function):
    def forward(self, y_hat, y, P, S=None, E=None):
        log_alpha = self._crf_forward(y_hat, P, S, E)
        log_beta = self._crf_backward(y_hat, P, S, E)
        
        log_Z = reduce_logsumexp(log_alpha[:, -1, :], axis=-1)
        
        transition_energy = P.gather(y)  # TODO
        emission_energy = torch.sum(y_hat.gather(y), asix=-1)
        log_M = transition_energy + emition_energy
        
        self.save_for_backward(y_hat, y, P, S, E, log_alpha, log_beta, log_Z, log_M)
        
        nll = np.sum(log_Z - log_M)
        return nll
    
    def backward(self, output_grad):
        y_hat, y, P, S, E, log_alpha, log_beat, log_Z, log_M = self.saved_variables
        
        y_grad = y_hat.new(*y_hat.size())
        P_grad = P.new(*P.size())
        if S is not None:
            S_grad = S.new(*S.size())
        else:
            S_grad = None
        if E is not None:
            E_grad = E.new(*E.size())
        else:
            E_grad = None
            
        y_grad = torch.exp(log_alpha + log_beta - log_Z)
        
        grad[i, j, k] -= np.exp(log_alpha[i - 1, j] + log_beta[i, k] - log_Z)
                if j == seq[i - 1] and k == seq[i]:
                    grad[i, j, k] += np.exp(-log_psi[i, j, k])
            
        return y_grad, P_grad, S_grad, E_grad
        
        
    def _crf_forward(self, input, P, S=None, E=None):
        batch_size, seq_len, voc = input.shape
        log_alpha = input.new(batch_size, seq_len, voc).fill_(-10000)
        
        if S is not None:
            log_alpha[:, 0, :] = S.unsqueeze(0).expand(batch_size, voc)
            
        for t in range(seq_len):
            energy = P.unsqueeze(0).expand(batch_size, voc, voc)  # transmit
            energy += input[:, t, :].unsqueeze(1).expand(batch_size, voc, voc)  # emit
            
            log_alpha_tm1 = log_alpha[:, t - 1, :].unsequeeze(1).expand(batch_size, voc, voc)
            
            log_alpha[:, t, :] = reduce_logsumexp(log_alpha_tm1 + energy, axis=-1)
            
        if E is not None:
            log_alpha[:, -1, :] += logsumexp(log_alpha[:, -1, :], E.unsqueeze(0).expand(batch_size, voc))

        nll = -reduce_logsumexp(log_alpha[:, -1, :])
        
        return log_alpha

    def _crf_backward(self, input, P, S=None, E=None):
        batch_size, seq_len, voc = input.shape
        log_beta = input.new(batch_size, seq_len, voc).fill_(-10000)
        
        if E is not None:
            log_beta[:, -1, :] = E.unsqueeze(0).expand(batch_size, voc)
        else:
            log_beta[:, -1, :] = 0
            
        for t in range(seq_len - 1, -1, -1):
            energy = P.unsqueeze(0).expand(batch_size, voc, voc)  # transmit
            energy += input[:, t, :].unsqueeze(1).expand(batch_size, voc, voc)  # emit
            
            log_beta_tp1 = log_beta[:, t + 1, :].unsequeeze(1).expand(batch_size, voc, voc)
            
            log_beta[:, t, :] = reduce_logsumexp(log_beta_tp1 + energy, axis=-1)
            
        if S is not None:
            log_beta[:, 0, :] += logsumexp(log_beta[:, 0, :], S.unsqueeze(0).expand(batch_size, voc))
            
        return log_beta    

IndentationError: unexpected indent (<ipython-input-12-474513284093>, line 41)

# References

## CRF
1. Sutton and McCuallum. [An Introduction to Conditional Random Fields](http://homepages.inf.ed.ac.uk/csutton/publications/crftut-fnt.pdf).
2. Michael Collins.[The Forward-Backward Algorithm](http://www.cs.columbia.edu/~mcollins/fb.pdf).
3. [Pytorch CRF Forward and Viterbi Implementation](http://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html).
4. [BiLSTM-CRF on PyTorch](https://github.com/kaniblu/pytorch-bilstmcrf).
5. Collobert. [Deep Learning for Efficient Discriminative Parsing](http://ronan.collobert.com/pub/matos/2011_parsing_aistats.pdf).
6. Collobert et al. [Natural Language Processing (Almost) from Scratch](http://ronan.collobert.com/pub/matos/2011_nlp_jmlr.pdf).
7. Huang et al. [Bidirectional LSTM-CRF Models for Sequence Tagging](https://arxiv.org/abs/1508.01991).

8. https://github.com/UKPLab/emnlp2017-bilstm-cnn-crf